In [9]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import pandas as pd

In [34]:
class Dataset(Dataset):
    def __init__(self, features_file, label_file):
        self.features = pd.read_csv(features_file)
        self.labels = pd.read_csv(label_file)

    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        tensor_features = torch.tensor(self.features.iloc[idx].values, dtype=torch.float)
        tensor_labels = torch.tensor(self.labels.iloc[idx].values, dtype=torch.float)

        return tensor_features, tensor_labels

In [35]:
dataset = Dataset('H1_AI_Dataset/Dataset1_Input.csv', 'H1_AI_Dataset/Dataset1_Labels.csv')
dataloader = DataLoader(dataset, batch_size=64, shuffle=False)

for features, labels in dataloader:  # This correctly unpacks each batch into features and labels
    print("Features:\n", features)
    print("Labels:\n", labels)
    print("Shape of features:", features.shape)  
    print("Shape of labels:", labels.shape)  
    break

Features:
 tensor([[0.0357, 0.0435, 0.0232, 0.0237, 0.0247, 0.0352, 0.0235, 0.0345],
        [0.0435, 0.0232, 0.0237, 0.0247, 0.0352, 0.0235, 0.0345, 0.0436],
        [0.0232, 0.0237, 0.0247, 0.0352, 0.0235, 0.0345, 0.0436, 0.0410],
        [0.0237, 0.0247, 0.0352, 0.0235, 0.0345, 0.0436, 0.0410, 0.0288],
        [0.0247, 0.0352, 0.0235, 0.0345, 0.0436, 0.0410, 0.0288, 0.0232],
        [0.0352, 0.0235, 0.0345, 0.0436, 0.0410, 0.0288, 0.0232, 0.0336],
        [0.0235, 0.0345, 0.0436, 0.0410, 0.0288, 0.0232, 0.0336, 0.0263],
        [0.0345, 0.0436, 0.0410, 0.0288, 0.0232, 0.0336, 0.0263, 0.0260],
        [0.0436, 0.0410, 0.0288, 0.0232, 0.0336, 0.0263, 0.0260, 0.0428],
        [0.0410, 0.0288, 0.0232, 0.0336, 0.0263, 0.0260, 0.0428, 0.0339],
        [0.0288, 0.0232, 0.0336, 0.0263, 0.0260, 0.0428, 0.0339, 0.0231],
        [0.0232, 0.0336, 0.0263, 0.0260, 0.0428, 0.0339, 0.0231, 0.0228],
        [0.0336, 0.0263, 0.0260, 0.0428, 0.0339, 0.0231, 0.0228, 0.0262],
        [0.0263, 0.0260, 0.

In [53]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=False)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [57]:
model = LSTMModel(1,128,2,6)
criterion = nn.MSELoss()  # For regression; change accordingly if it's a classification task
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [58]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

LSTMModel(
  (lstm): LSTM(1, 128, num_layers=2)
  (fc): Linear(in_features=128, out_features=6, bias=True)
)

In [59]:
# Training loop
num_epochs = 4
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0
    for batch_idx, (data, targets) in enumerate(dataloader):
        data, targets = data.to(device), targets.to(device)

        print(data.shape)
        print(labels.shape)

        # Forward pass
        outputs = model(data)
        loss = loss_function(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}')

torch.Size([64, 8])
torch.Size([64, 6])


RuntimeError: For unbatched 2-D input, hx and cx should also be 2-D but got (3-D, 3-D) tensors